In [1]:
import pandas as pd
import requests
import time

In [2]:
# Load the cleaned DataFrame from the CSV file
books_df = pd.read_csv('books_toscrape.csv')

In [3]:
books_df

,Title,Price,Availability,Rating,Link
0,A Light in the Attic,Â£51.77,In stock,Three,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,Â£53.74,In stock,One,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,Â£50.10,In stock,One,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,Â£47.82,In stock,Four,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five,http://books.toscrape.com/catalogue/sapiens-a-...
...,...,...,...,...,...
495,Unreasonable Hope: Finding Faith in the God Wh...,Â£46.33,In stock,Two,http://books.toscrape.com/catalogue/unreasonab...
496,Under the Tuscan Sun,Â£37.33,In stock,Three,http://books.toscrape.com/catalogue/under-the-...
497,Toddlers Are A**holes: It's Not Your Fault,Â£25.55,In stock,One,http://books.toscrape.com/catalogue/toddlers-a...
498,The Year of Living Biblically: One Man's Humbl...,Â£34.72,In stock,One,http://books.toscrape.com/catalogue/the-year-o...


In [4]:
# Google Books API endpoint
google_books_api = "https://www.googleapis.com/books/v1/volumes"
i=0
book_details = pd.DataFrame()

In [5]:
# Function to fetch book details from Google Books API
def fetch_google_books_details(title,i):
    # Define the parameters for the API request
    print(i , title)
    params = {
        'q': title,
        'maxResults': 1,
        'printType': 'books'
    }
    
    # Send the request to the Google Books API
    response = requests.get(google_books_api, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'items' in data:

            volume_info = data['items'][0]['volumeInfo']
            print(volume_info)
            return {
                'Title':title,
                'Authors': ', '.join(volume_info.get('authors', [])),
                'Publisher': volume_info.get('publisher', ''),
                'Published Date': volume_info.get('publishedDate', ''),
                'Description': volume_info.get('description', ''),
                'ISBN-13': next((identifier['identifier'] for identifier in volume_info.get('industryIdentifiers', []) if identifier['type'] == 'ISBN_13'), '')
            }
    return {}

In [6]:

# Create lists to store the fetched details
titles = []
authors = []
publishers = []
published_dates = []
descriptions = []
isbn13s = []


In [7]:

# Fetch details for each book title

for index, row in books_df.iterrows():
    i+=1
    title = row['Title']
    details = fetch_google_books_details(title,i)
    
    # Append the details to the respective lists
    titles.append(details.get('Title'))
    authors.append(details.get('Authors', ''))
    publishers.append(details.get('Publisher', ''))
    published_dates.append(details.get('Published Date', ''))
    descriptions.append(details.get('Description', ''))
    isbn13s.append(details.get('ISBN-13', ''))
    
    # Pause for a short period to avoid hitting the API rate limit
    time.sleep(1)

1 A Light in the Attic
{'title': 'A Light in the Attic Book and CD', 'authors': ['Shel Silverstein'], 'publisher': 'HarperCollins', 'publishedDate': '2001-10-09', 'description': "Last night while I lay thinking here Some Whatifs crawled inside my ear And pranced and partied all night long And sang their same old Whatif song: Whatif I flunk that test? Whatif green hair grows on my chest? Whatif nobody likes me? Whatif a bolt of lightning strikes me?... To celebrate its 20th anniversary, Shel Silverstein's A Light in the Attic is now available in a special edition containing the classic hardcover book and a CD of highlights from his Grammy Award-winning album. Here in the attic of Shel Silverstein you will find Backward Bill, Sour Face Ann, the Meehoo with an Exactlywatt, and the Polar Bear in the Frigidaire. You will talk with Broiled Face, and find out what happens when Somebody steals your knees, you get caught by the Quick-Digesting Gink, a Mountain snores, and They Put a Brassiere o

In [8]:
# Add the fetched details to the DataFrame
book_details['Title']= titles
book_details['Authors'] = authors
book_details['Publisher'] = publishers
book_details['Published Date'] = published_dates
book_details['Description'] = descriptions
book_details['ISBN-13'] = isbn13s

In [9]:
book_details

,Title,Authors,Publisher,Published Date,Description,ISBN-13
0,A Light in the Attic,Shel Silverstein,HarperCollins,2001-10-09,Last night while I lay thinking here Some What...,9780066236179
1,Tipping the Velvet,Sarah Waters,Virago,2011-02-03,From the oyster huts of Whitstable to the musi...,9780748129324
2,Soumission,Michel Houellebecq,Macmillan,2015-10-20,"In a near-future France, Franocois, a middle-a...",9780374271572
3,Sharp Objects,Gillian Flynn,Crown,2018-06-12,"NOW AN HBO® LIMITED SERIES STARRING AMY ADAMS,...",9780525575740
4,Sapiens: A Brief History of Humankind,Yuval Noah Harari,Harper Collins,2015-02-10,New York Times Bestseller A Summer Reading Pic...,9780062316103
...,...,...,...,...,...,...
495,Unreasonable Hope: Finding Faith in the God Wh...,Chad Veach,Thomas Nelson,2016-03-08,“Where was God when ____? How could God allow ...,9780718038366
496,Under the Tuscan Sun,Frances Mayes,Crown,2003-08-26,#1 NEW YORK TIMES BESTSELLER • The beloved mem...,9780767917452
497,Toddlers Are A**holes: It's Not Your Fault,Bunmi Laditan,Workman Publishing Company,2015-04-07,Toddler a**holery is a normal part of human de...,9780761185758
498,The Year of Living Biblically: One Man's Humbl...,A. J. Jacobs,Simon and Schuster,2008-09-09,Documents the author's quest to live one year ...,9780743291484


In [11]:
# Save the enriched DataFrame to a new CSV file
book_details.to_csv('books_detail.csv', index=False)

print(' DataFrame saved as books_detail.csv')

 DataFrame saved as books_detail.csv


In [12]:
# Display the first few rows of the enriched DataFrame
books_df.head()

,Title,Price,Availability,Rating,Link
0,A Light in the Attic,Â£51.77,In stock,Three,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,Â£53.74,In stock,One,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,Â£50.10,In stock,One,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,Â£47.82,In stock,Four,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five,http://books.toscrape.com/catalogue/sapiens-a-...


In [13]:
books_df

,Title,Price,Availability,Rating,Link
0,A Light in the Attic,Â£51.77,In stock,Three,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,Â£53.74,In stock,One,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,Â£50.10,In stock,One,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,Â£47.82,In stock,Four,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five,http://books.toscrape.com/catalogue/sapiens-a-...
...,...,...,...,...,...
495,Unreasonable Hope: Finding Faith in the God Wh...,Â£46.33,In stock,Two,http://books.toscrape.com/catalogue/unreasonab...
496,Under the Tuscan Sun,Â£37.33,In stock,Three,http://books.toscrape.com/catalogue/under-the-...
497,Toddlers Are A**holes: It's Not Your Fault,Â£25.55,In stock,One,http://books.toscrape.com/catalogue/toddlers-a...
498,The Year of Living Biblically: One Man's Humbl...,Â£34.72,In stock,One,http://books.toscrape.com/catalogue/the-year-o...
